# IMDB 영화 리뷰 감성 분석 (Multi-hot Encoding)

## 1. 라이브러리 임포트

In [ ]:
import requests
import subprocess
import re
import string
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import os, pathlib, shutil, random
import keras

## 2. 데이터 다운로드 및 준비

IMDB 영화 리뷰 데이터셋을 다운로드하고, 압축을 해제한 후, 훈련 데이터의 일부를 검증 데이터로 분리합니다.

In [ ]:
# 데이터 다운로드 함수
def download():
    url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
    file_name = "aclImdb_v1.tar.gz"
    if not os.path.exists(file_name):
        print("Downloading data...")
        response = requests.get(url, stream=True)
        with open(file_name, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print("Download complete!")
    else:
        print("Data already downloaded.")

# 압축 해제 함수
def release():
    if not os.path.exists("aclImdb"):
        print("Extracting files...")
        subprocess.run(["tar", "-xvzf", "aclImdb_v1.tar.gz"], shell=True)
        # unsup 폴더 제거 (라벨이 없으므로 사용 안함)
        if os.path.exists("aclImdb/train/unsup"):
            shutil.rmtree("aclImdb/train/unsup")
        print("Extraction complete.")
    else:
        print("Data already extracted.")

# 훈련/검증 데이터 분리 함수
def labeling(): 
    base_dir = pathlib.Path("aclImdb")
    val_dir = base_dir / "val"
    train_dir = base_dir / "train"
    if not val_dir.exists():
        print("Creating validation set...")
        for category in ("neg", "pos"):
            os.makedirs(val_dir / category)
            files = os.listdir(train_dir / category)
            random.Random(1337).shuffle(files)
            num_val_samples = int(0.2 * len(files))
            val_files = files[-num_val_samples:]
            for fname in val_files:
                shutil.move(train_dir / category / fname, val_dir / category / fname)
        print("Validation set created.")
    else:
        print("Validation set already exists.")

In [ ]:
# 최초 실행 시에만 주석 해제
# download()
# release()
# labeling()

## 3. 데이터셋 로드

`text_dataset_from_directory`를 사용하여 디렉토리 구조를 기반으로 훈련, 검증, 테스트 데이터셋을 생성합니다.

In [ ]:
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size=batch_size)

### 데이터셋 구조 확인

In [ ]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

## 4. 텍스트 벡터화 (Multi-hot Encoding)

텍스트 데이터를 수치형 데이터로 변환합니다. 여기서는 **Multi-hot Encoding** 또는 **Bag-of-Words(단어 가방)** 방식을 사용합니다. 이 방식은 문장에서 단어의 순서는 무시하고, 어휘 사전에 있는 단어의 등장 유무만 벡터에 표시합니다.

- `max_tokens=20000`: 어휘 사전에 포함할 단어의 최대 개수를 20,000개로 제한합니다. (빈도수 기준)
- `output_mode='multi_hot'`: 각 텍스트를 20,000차원의 벡터로 변환합니다. 텍스트에 어휘 사전에 있는 단어가 포함되어 있으면 해당 인덱스의 값을 1로, 그렇지 않으면 0으로 설정합니다.

In [ ]:
text_vectorization = TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot"
)

### 어휘 사전 생성 및 데이터셋 변환

훈련 데이터셋의 텍스트만을 사용하여 어휘 사전을 생성(`adapt`)하고, 이 사전을 기준으로 모든 데이터셋(훈련, 검증, 테스트)을 Multi-hot 벡터로 변환합니다.

In [ ]:
# 훈련 데이터에서 텍스트만 추출하여 어휘 사전을 생성합니다.
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

# 모든 데이터셋에 벡터화 레이어를 적용합니다.
# num_parallel_calls를 사용하여 전처리 속도를 높입니다.
binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=tf.data.AUTOTUNE)
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=tf.data.AUTOTUNE)
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=tf.data.AUTOTUNE)

### 벡터화된 데이터 확인

In [ ]:
print("--- 벡터화 후 ---")
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape) # (batch_size, max_tokens)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0][:20]:", inputs[0][:20]) # 첫 번째 리뷰 벡터의 앞 20개 값
    print("targets[0]:", targets[0])
    break

## 5. 모델 구축 및 컴파일

간단한 완전 연결 신경망(Dense) 모델을 생성합니다.

In [ ]:
from keras import layers, models

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

model = get_model()
model.summary()

## 6. 모델 훈련

- `ModelCheckpoint`: 검증 세트에서 가장 좋은 성능을 내는 모델을 파일로 저장합니다.
- `.cache()`: 데이터셋을 메모리에 캐시하여 다음 에포크부터는 전처리 과정을 생략하고 훈련 속도를 높입니다. (메모리에 올릴 수 있을 만큼 데이터셋이 작을 때 유용)

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras", save_best_only=True)
]

model.fit(
    binary_1gram_train_ds.cache(),
    validation_data=binary_1gram_val_ds.cache(),
    epochs=10,
    callbacks=callbacks
)

## 7. 모델 평가

훈련 과정에서 저장된 최적의 모델을 불러와 테스트 데이터셋으로 최종 성능을 평가합니다.

In [ ]:
model = models.load_model("binary_1gram.keras")
print(f"테스트셋 정확도: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")